# Model Training

In [1]:
pwd

'/home/wsuser/work'


# Importing libraries

In [2]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten, Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# Image Data Argumentation

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='u-se74_mx-eSsqY4pIo8Aw_hmv4uHDH2jk1LJuQtPaNB',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'agesturebasedtoolforsterilebrowsi-donotdelete-pr-gspuvwpzl2ogtz'
object_key = 'data.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

# Loading Our Data And Perform Data Argumentation

In [7]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/data/train', target_size=(64, 64), batch_size=3, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('/home/wsuser/work/data/test', target_size=(64, 64), batch_size=3, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [8]:
print(x_train.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


# Initializing The Model

In [9]:
model=Sequential()

# Adding CNN Layers

In [10]:
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [11]:
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
model.add(Flatten())

# Adding Dense Layers

In [13]:
model.add(Dense(units=512, activation='relu'))

In [14]:
model.add(Dense(units=6, activation='softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 512)               3

# Model Compilation


In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Fitting

In [17]:
model.fit_generator(x_train,
                    steps_per_epoch =594/3,
                    epochs=25,
                    validation_data=x_test,
                    validation_steps=30/3)

/tmp/wsuser/ipykernel_4903/1173897450.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,


Epoch 1/25
198/198 [==============================] - 13s 65ms/step - loss: 1.2472 - accuracy: 0.4933 - val_loss: 1.1850 - val_accuracy: 0.5667
Epoch 2/25
198/198 [==============================] - 13s 63ms/step - loss: 0.6108 - accuracy: 0.7643 - val_loss: 0.5540 - val_accuracy: 0.8000
Epoch 3/25
198/198 [==============================] - 13s 65ms/step - loss: 0.4070 - accuracy: 0.8468 - val_loss: 0.2637 - val_accuracy: 0.9333
Epoch 4/25
198/198 [==============================] - 13s 65ms/step - loss: 0.3024 - accuracy: 0.8788 - val_loss: 0.2726 - val_accuracy: 0.9000
Epoch 5/25
198/198 [==============================] - 13s 64ms/step - loss: 0.2229 - accuracy: 0.9209 - val_loss: 0.3331 - val_accuracy: 0.9000
Epoch 6/25
198/198 [==============================] - 13s 64ms/step - loss: 0.1963 - accuracy: 0.9276 - val_loss: 0.2571 - val_accuracy: 0.9333
Epoch 7/25
198/198 [==============================] - 12s 63ms/step - loss: 0.1027 - accuracy: 0.9596 - val_loss: 0.2199 - val_accuracy:

# Save The Model

In [18]:
model.save('gesture.h5')

# Save The Model in tar file format

In [19]:
!tar -zcvf Gesture.tgz gesture.h5

gesture.h5


In [20]:
ls

data/  gesture.h5  Gesture.tgz


# IBM Deployment

In [21]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 9.6 MB/s eta 0:00:01


In [22]:
from ibm_watson_machine_learning import APIClient

API_KEY = "56Us00azghgSCg275Hjgqk9tXGZSnfziZmWdKi57d-yw"

wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(wml_credentials)

In [23]:
client=APIClient(wml_credentials)

In [24]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [26]:
space_uid = guid_from_space_name(client, 'A_Gesture based tool for radiology images')
print("Space UID: ", space_uid)

Space UID:  fc5179cd-b6b9-4112-a5d8-3a39b8faec1c


In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [29]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [31]:
model_details = client.repository.store_model(model="Gesture.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [32]:
model_id

'a22758ac-cdec-440e-9291-756f8faf579e'

In [33]:
client.repository.download(model_id, "model.tar.gz")

Successfully saved model content to file: 'model.tar.gz'


'/home/wsuser/work/model.tar.gz'